<a href="https://colab.research.google.com/github/mgetsova/kaggle/blob/main/IceCube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper functions for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

!pip install opendatasets
!pip install pandas

2.0.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pathlib import Path
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union
import numpy as np
import pandas as pd
import torch

from drive.MyDrive.IceCube.ice_transparency import ice_transparency
from drive.MyDrive.IceCube.prepare_sensors import prepare_sensors

INPUT_PATH = Path("drive/MyDrive/IceCube")
TRANSPARENCY_PATH = INPUT_PATH / "ice_transperancy.txt"
#FULL_TRAIN_META_PATH = INPUT_PATH / "train_meta.parquet"
TRAIN_PATH = INPUT_PATH / "train"
STARTER_PULSE_PATH = TRAIN_PATH / 'pulses'
STARTER_META_PATH = TRAIN_PATH / 'meta'

BATCH_FILE = 1

meta = pd.read_parquet(STARTER_META_PATH / f"batch_{BATCH_FILE}.parquet", columns=["event_id", 'azimuth', 'zenith'], engine="pyarrow", use_threads=True)
sensor_df = prepare_sensors(INPUT_PATH / "sensor_geometry.csv")
f_scattering, f_absorption = ice_transparency(TRANSPARENCY_PATH)
event_ids = meta.event_id
y = meta[['zenith', 'azimuth']].reset_index(drop=True)
xx = np.cos(y['azimuth']) * np.sin(y['zenith'])
yy = np.sin(y['azimuth']) * np.sin(y['zenith'])
zz = np.cos(y['zenith'])
y['dir_x'] = xx
y['dir_y'] = yy
y['dir_z'] = zz


y.head()

,zenith,azimuth,dir_x,dir_y,dir_z
0,2.087498,5.029555,0.271161,-0.826088,-0.494015
1,1.549686,0.417742,0.913804,0.405607,0.021108
2,2.401942,1.160466,0.268879,0.618078,-0.738704
3,0.759054,5.845952,0.623491,-0.291423,0.725488
4,0.939117,0.653719,0.640648,0.490794,0.590501


In [14]:
y.loc[0][2:].values
#y.loc[0,:].values

array([ 0.27116069, -0.82608801, -0.49401465])

Zenith ranges from 0 to pi. 
<br>
Azimuth ranges from 0 to 2pi.

**for sensors:**
<br>
118 x coords 
<br>
117 y coords
<br>
4975 z coords
<br>
86 strings (as described in paper)
<br>
qe is assumed to be the same as relative DOM efficiency (where bottom 50 detectors have a higher value) 
<br> <br>
above are scaled as in graphnet model for IceCube86 ...coords divided by 500, strings go from 0-85 (presumably, also don't use in training) and eq is two numbers, either -1 or 0.4 

In [3]:
sensor_df['qe'].value_counts()

-1.0    4761
 0.4     399
Name: qe, dtype: int64

In [4]:
import torch
#from torch.utils.data import Dataset
from torch_geometric.nn import knn_graph
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader

class IceCubeDataset(Dataset):
    def __init__(
        self,
        batch_id,
        event_ids,
        PATH_TO_BATCH_FILES,
        f_scattering,
        f_absorption,
        sensor_df,
        y,
        pulse_limit=300,
        transform=None,
        pre_transform=None,
        pre_filter=None,
    ):
        super().__init__(transform, pre_transform, pre_filter)
        self.event_ids = event_ids
        self.batch_df = pd.read_parquet(PATH_TO_BATCH_FILES / f"batch_{batch_id}.parquet")
        self.sensor_df = sensor_df
        self.pulse_limit = pulse_limit
        self.f_scattering = f_scattering
        self.f_absorption = f_absorption
        self.y = y
        #ice_transparency(TRANSPARENCY_PATH)

        self.batch_df["time"] = (self.batch_df["time"] - 1.0e04) / 3.0e4
        self.batch_df["charge"] = np.log10(self.batch_df["charge"]) / 3.0
        self.batch_df["auxiliary"] = self.batch_df["auxiliary"].astype(int) - 0.5

    def len(self):
        return len(self.event_ids)

    def get(self, idx):
        event_id = self.event_ids[idx]
        event = self.batch_df.loc[event_id]

        event = pd.merge(event, self.sensor_df, on="sensor_id")

        x = event[["x", "y", "z", "time", "charge", "qe", "auxiliary"]].values
        x = torch.tensor(x, dtype=torch.float32)
        data = Data(x=x, n_pulses=torch.tensor(x.shape[0], dtype=torch.int32))

        # Add ice transparency data
        z = data.x[:, 2].numpy()
        scattering = torch.tensor(self.f_scattering(z), dtype=torch.float32).view(-1, 1)
        # absorption = torch.tensor(self.f_absorption(z), dtype=torch.float32).view(-1, 1)

        data.x = torch.cat([data.x, scattering], dim=1)

        # Downsample the large events
        if data.n_pulses > self.pulse_limit:
            data.x = data.x[np.random.choice(data.n_pulses, self.pulse_limit)]
            data.n_pulses = torch.tensor(self.pulse_limit, dtype=torch.int32)

         # Builds graph from the k-nearest neighbours.
        data.edge_index = knn_graph(
            data.x[:, [0, 1, 2]],  # x, y, z
            k=8,
            batch=None,
            loop=False
        )

        if self.y is not None:
            #y = self.y.loc[idx, :].values
            y = self.y.loc[idx][2:].values # directions
            y = torch.tensor(y, dtype=torch.float32)
            data.y = y

        return data

In [5]:
dataset = IceCubeDataset(BATCH_FILE, list(event_ids), STARTER_PULSE_PATH, f_scattering, f_absorption, sensor_df, y)
#dataset.len()
dataset.get(0).x.shape
#dataset.get(0).x.shape[1]

torch.Size([61, 8])

In [6]:
from torch_geometric.utils.homophily import homophily

def calculate_xyzt_homophily(x, edge_index, batch):
    """Calculate xyzt-homophily from a batch of graphs.

    Homophily is a graph scalar quantity that measures the likeness of
    variables in nodes. Notice that this calculator assumes a special order of
    input features in x.

    Returns:
        Tuple, each element with shape [batch_size,1].
    """
    hx = homophily(edge_index, x[:, 0], batch).reshape(-1, 1)
    hy = homophily(edge_index, x[:, 1], batch).reshape(-1, 1)
    hz = homophily(edge_index, x[:, 2], batch).reshape(-1, 1)
    ht = homophily(edge_index, x[:, 3], batch).reshape(-1, 1)
    return hx, hy, hz, ht

In [7]:
from torch_geometric.nn import EdgeConv
from torch_geometric.nn.pool import knn_graph # differtent from one above????
from typing import Any, Callable, List, Optional, Sequence, Tuple, Union
from torch import LongTensor, Tensor
from torch_geometric.typing import Adj

class DynEdgeConv(EdgeConv):
    """Dynamical edge convolution layer."""

    def __init__(
        self,
        nn: Callable,
        device,
        aggr: str = "max",
        nb_neighbors: int = 8,
        features_subset: Optional[Union[Sequence[int], slice]] = None,
        **kwargs: Any,
    ):
        """Construct `DynEdgeConv`.
        Args:
            nn: The MLP/torch.Module to be used within the `EdgeConv`.
            aggr: Aggregation method to be used with `EdgeConv`.
            nb_neighbors: Number of neighbours to be clustered after the
                `EdgeConv` operation.
            features_subset: Subset of features in `Data.x` that should be used
                when dynamically performing the new graph clustering after the
                `EdgeConv` operation. Defaults to all features.
            **kwargs: Additional features to be passed to `EdgeConv`.
        """
        # Check(s)
        if features_subset is None:
            features_subset = slice(None)  # Use all features
        assert isinstance(features_subset, (list, slice))

        # Base class constructor
        super().__init__(nn=nn, aggr=aggr, **kwargs)

        # Additional member variables
        self.device = device
        self.nb_neighbors = nb_neighbors
        self.features_subset = features_subset

    def forward(
        self, x: Tensor, edge_index: Adj, batch: Optional[Tensor] = None
    ) -> Tensor:

        """Forward pass."""
        # Standard EdgeConv forward pass
        x = super().forward(x, edge_index)
        # Recompute adjacency
        edge_index = knn_graph(x=x[:, self.features_subset], k=self.nb_neighbors, 
                               batch=batch).to(self.device)

        return x, edge_index

In [8]:
def define_mlp(nb_in, nb_hidden, nb_out, activation):
    return torch.nn.Sequential(torch.nn.Linear(nb_in, nb_hidden),
                               activation,
                               torch.nn.Linear(nb_hidden, nb_out),
                               activation)
    

In [9]:
import torch.nn as nn
from torch_scatter import scatter_max, scatter_mean, scatter_min, scatter_sum

GLOBAL_POOLINGS = {
    "min": scatter_min,
    "max": scatter_max,
    "sum": scatter_sum,
    "mean": scatter_mean,
}

class MyGNN(nn.Module):
    def __init__(self, nb_inputs, features_subset, out_dim, device = 'cpu', nb_neighbors = 8, global_pooling_schemes = ['mean', 'min', 'max', 'sum'], 
                 add_global_variables_after_pooling = True, num_global_variables = 5):
        super().__init__()
        self.activation = torch.nn.LeakyReLU()
        self.device = device

        self.conv_layer1 = DynEdgeConv(define_mlp(nb_inputs*2, 128, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer2 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer3 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)
        self.conv_layer4 = DynEdgeConv(define_mlp(256*2, 336, 256, self.activation), self.device, aggr="add", nb_neighbors=8, features_subset=features_subset)

        self.post_processing = define_mlp(256*4 + nb_inputs, 336, 256, self.activation)

        nb_poolings = (len(global_pooling_schemes) if global_pooling_schemes else 1)
        nb_latent_features = 256 * nb_poolings
        if add_global_variables_after_pooling:
            nb_latent_features += num_global_variables

        #self.readout = torch.nn.Sequential(torch.nn.Linear(nb_latent_features, 128), self.activation)
        self.readout = define_mlp(nb_latent_features, 128, out_dim, self.activation)
        
        self._global_pooling_schemes = global_pooling_schemes
        self._add_global_variables_after_pooling = add_global_variables_after_pooling

    def _global_pooling(self, x: Tensor, batch: LongTensor) -> Tensor:
      """Perform global pooling."""
      assert self._global_pooling_schemes
      pooled = []
      for pooling_scheme in self._global_pooling_schemes:
        pooling_fn = GLOBAL_POOLINGS[pooling_scheme]
        pooled_x = pooling_fn(x, index=batch, dim=0)
        if isinstance(pooled_x, tuple) and len(pooled_x) == 2:
          pooled_x, _ = pooled_x
        pooled.append(pooled_x)

      return torch.cat(pooled, dim=1)

    def _calculate_global_variables(self, x: Tensor, edge_index: LongTensor, batch: LongTensor, 
                                    *additional_attributes: Tensor,) -> Tensor:
      """Calculate global variables."""
      # Calculate homophily (scalar variables)
      h_x, h_y, h_z, h_t = calculate_xyzt_homophily(x, edge_index, batch)
      # Calculate mean features
      global_means = scatter_mean(x, batch, dim=0)
      # Add global variables
      # global_variables = torch.cat([global_means, h_x, h_y, h_z, h_t] + [attr.unsqueeze(dim=1) for attr in additional_attributes], dim=1,)
      global_variables = torch.cat([h_x, h_y, h_z, h_t] + [attr.unsqueeze(dim=1) for attr in additional_attributes], dim=1,)


      return global_variables

    def forward(self, data: Data) -> Tensor:
      """Apply learnable forward pass."""
      # Convenience variables
      x, edge_index, batch = data.x, data.edge_index, data.batch
      global_variables = self._calculate_global_variables(x, edge_index, 
                                                          batch, torch.log10(data.n_pulses))

      # Distribute global variables out to each node
      if not self._add_global_variables_after_pooling:
        distribute = (batch.unsqueeze(dim=1) == torch.unique(batch).unsqueeze(dim=0)).type(torch.float) 
        global_variables_distributed = torch.sum(distribute.unsqueeze(dim=2) * 
                                                 global_variables.unsqueeze(dim=0), dim=1)
        x = torch.cat((x, global_variables_distributed), dim=1)

      # convolutions
      state_graphs = [x]
      conv_layers = [self.conv_layer1, self.conv_layer2, self.conv_layer3, self.conv_layer4]
      for layer in conv_layers:
        x, edge_index = layer(x, edge_index, batch)
        state_graphs.append(x)

      x = torch.cat(state_graphs, dim=1)
      # post processing
      x = self.post_processing(x)

      # global pooling
      if self._global_pooling_schemes:
        x = self._global_pooling(x, batch=batch)
        if self._add_global_variables_after_pooling:
          x = torch.cat([x, global_variables], dim=1)

      # readout layer
      x = self.readout(x)
      #x = torch.sigmoid(x) * np.pi
      return x



In [10]:
def vMF_loss(n_pred, n_true, eps = 1e-8):
    """  von Mises-Fisher Loss: n_true is unit vector ! """
    kappa = torch.norm(n_pred, dim=0)        
    logC  = -kappa + torch.log( ( kappa+eps )/( 1-torch.exp(-2*kappa)+2*eps ) )
    return -((n_true*n_pred).sum(dim=0) + logC).mean()

def make_coord_vec(angles):
    azimuth = angles[0]
    zenith = angles[1]

    x = torch.cos(azimuth) + torch.sin(zenith)
    y = torch.sin(azimuth) + torch.sin(zenith)
    z = torch.cos(zenith)
    return torch.cat([x, y, z], 0)

In [11]:
compute_knn_with = [0, 1, 2] # x, y, z
model = MyGNN(8, compute_knn_with, 3, device='cuda')
model

MyGNN(
  (activation): LeakyReLU(negative_slope=0.01)
  (conv_layer1): DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  ))
  (conv_layer2): DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=512, out_features=336, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=336, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  ))
  (conv_layer3): DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=512, out_features=336, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=336, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  ))
  (conv_layer4): DynEdgeConv(nn=Sequential(
    (0): Linear(in_features=512, out_features=336, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=336, out_featu

In [17]:
epochs = 1
batchsize = 200
#loss_fn = vMF_loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using ', device)
model = model.to(device)

using  cuda


In [18]:
batch_ids = [BATCH_FILE, BATCH_FILE] # for now

from tqdm import tqdm
from torch_geometric.loader import DataLoader
for i, b in enumerate(batch_ids):
    ##### for more general training later:

    # event_ids = meta[meta["batch_id"] == b]["event_id"].tolist()
    # y = meta[meta["batch_id"] == b][['zenith', 'azimuth']].reset_index(drop=True)
    # dataset = IceCubeDataset(
    #    b, event_ids, sensors, mode='train', y=y,
    # )

    ##### already constructed a dataset for one batch further up

    # just one batch dataset
    if i >= 1:
        break

    train_len = int(0.7*dataset.len())
    train_loader = DataLoader(dataset[:train_len], batch_size=batchsize, shuffle=True) # shuffle data every epoch
    val_loader = DataLoader(dataset[train_len:], batch_size=batchsize, shuffle=False)
    
    print(f'batch file = {i}')
    for epoch_num in range(epochs):
        print(f'epoch = {epoch_num}')
        total_loss_train = 0
        model.train()
        for sample_batched in tqdm(train_loader, desc='train'):
            sample_batched = sample_batched.to(device)
            # do one forward pass
            outputs = model(sample_batched)
            # label = sample_batched.y.reshape(-1, 2).to(device)
            label = sample_batched.y.reshape(-1, 3).to(device)
            #break
            # calculate loss (per sample batch)
            
            loss = vMF_loss(outputs, label)
            #loss = nn.L1Loss(outputs, label)
            total_loss_train += loss.cpu().item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        total_loss_val = 0
        model.eval()
        with torch.no_grad():
            for sample_batched in tqdm(val_loader, desc='val'):
                sample_batched = sample_batched.to(device)
                outputs = model(sample_batched)
                # label = sample_batched.y.reshape(-1, 2).to(device)
                label = sample_batched.y.reshape(-1, 3).to(device)
                loss = vMF_loss(outputs, label)
                #loss = nn.L1Loss(outputs, label)
                total_loss_val += loss.cpu().item()

        print(f'epoch[{epoch_num}]', total_loss_train / train_len, total_loss_val / (dataset.len() - train_len))
        torch.save(model, f"drive/MyDrive/IceCube/models/third_model_epoch{epoch_num}.pt")
        
        

torch.save(model, "drive/MyDrive/IceCube/models/third_model.pt")

batch file = 0
epoch = 0


val: 100%|██████████| 300/300 [04:44<00:00,  1.05it/s]

epoch[0] 0.0029979367303096557 0.0017303050362194577


In [ ]:
def infer(model, loader, device="cpu"):
    model.to(device)
    model.eval()

    predictions = []
    with torch.no_grad():
      for batch in loader:
        batch = batch.to(device)
        pred_angles = model(batch)
        predictions.append(pred_angles.cpu())

    return torch.cat(predictions, 0)